# Tutorial on Data Science in Astrophysics - EASW9

##### Prepared by Kono Kai, Shuntaro Yoshida and Suchetha Cooray ($\Omega$ Lab, NU)

### The point of this tutorial is NOT to make you data scientists in 1.5 hrs. This tutorial will hopefully give you the first steps for you to experience "some" data science techniques. 

We will cover the following areas of data science
- Data input
- Simple data handling
- Data visualization
- Data analysis with linear regression (example with a package)
- Parameter estimation (linear regression) with Markov chain Monte Carlo (MCMC)

### For data science, Python with jupyter notebooks are a very convinient tool. Therefore, we would like to demonstrate how to use them.

### Firstly, why jupyter notebooks?

From the official documentation,

*Notebook documents (or “notebooks”, all lower case) are documents produced by the Jupyter Notebook App, which contain both computer code (e.g. python) and rich text elements (paragraph, equations, figures, links, etc…). Notebook documents are both human-readable documents containing the analysis description and the results (figures, tables, etc..) as well as executable documents which can be run to perform data analysis.*

https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html#notebook-document

With that out of the way, let's start data science!

## Today we will work with data from a galaxy survey and do some simple data analysis.

If you get an "ImportError", you might be missing some libraries.
In that case, you should run the following lines on a new cell to install those libraries:

> if you can use conda -> !conda install {library name(e.g. scipy, tqdm)} --yes

> if you can use pip -> !pip3 install {library name(e.g. scipy, tqdm)}

## 1) Read the data 

Python is easy to read a datafile with pandas.

"pandas" is a library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language

## 2) Simple data handling

Pandas makes it easier to see "what" kind of data is in the data frame compared to a "traditional" array using numpy.

## 3) Data visualization

We can first try to understand the redshift distribution of our galaxy sample. The best for this purpose is a histogram.

Now let's try plotting a 2D scatter plot with two data columns. Let us say we choose stellar mass ($M_*$) and luminosity in the K-band ($L_K$) in log space.

Here, the distribution of $M_*{\mbox -}L_K$ seems to have a linear relation with two parameters $a$ and $b$ which correspond to slope and intercept respectively.

$$
\log (L_K)=a\log(M_*/M_{\odot})+b
$$

We have considered the whole sample. However, we are now interested only in star forming galaxies (SFGs). We limit our sample to galaxy with the "SFG" flag equal to one. 

By visual inspection, the linear relationship appears to be much more apparent now.

### Let us try linear least-squares regression for these sets of measurements!

## 4) Linear Regression

Let us try two methods to do regression.

First method is using the `scipy.stats` module. This module contains the `linregress`, function for linear least-squares regression.

The second method is by using `scipy.optimize.curve_fit`. This method allows fitting any user defined function. 

Simple as that! We get similar results from both the methods. Let us plot the `lingress` result on our scatter plot.

## 5) Parameter estimation by Markov Chain Monte Carlo

The estimated slope and intercept are just parameters. **We can use a Bayesian approach and use Markov Chain Monte Carlo (MCMC) to determine our parameters and their posterior distributions**.

Most important advantage of using a bayesian approach is that we can encorporate a prior for our parameter estimates. As the problems get more complicated, priors are crucial in guiding to acceptable results. 

Let us try to use MCMC to find the parameters of our linear model to $M_*{\mbox -}L_K$ data.

Firstly, we need to assume a prior distribution of parameters.

In this tutorial, we assume that the prior distribution of intercept and slope is normal distribution with mean $\mu=0.5$ and standard deviation $\sigma=0.5$.

$$
\mathcal{N}(\mu,\sigma)=\frac{1}{\sqrt{2\pi \sigma^2}}\exp{\biggl(-\frac{(x-\mu)^2}{2\sigma^2}\biggr)}
$$

The likelihood function is obtained as below. 

$$
L ( p ) = \prod _ { i = 1 } ^ { n } f \left( x _ { i } ; p \right)
$$

For computational reasons, the log-likelihood is often used.  

Then the posterior distribution for each step can be calculated using the likelihood and prior probability.

The posterior distribution can be used to estimate our parameters. In this case from our prior and the liklihoods, we expect the posterior to have normal distributions with the mean as the best estimates of the parameters.

### Here onwards, we perform the Markov chain Monte Carlo (MCMC). 

MCMC is a method for obtaining a sequence of random samples from a probability distribution from which direct sampling is difficult. This sequence can be used to approximate the distribution (e.g. to generate a histogram) or to compute an integral (e.g. an expected value). In this case, we want the posterior distributions of the parameters. Sequential samples are simple Markov chains, that are random walks that depend only on the last location.

We will use the most famous MCMC algorithm known as the **Metropolis-Hastings (MH) algorithm**.

**The goal is to design the Markov chain to converge to the desired posterior distribution.**

Therefore, as more and more sample values are produced, the distribution of values more closely approximates the desired distribution (i.e. posterior distribution).

The algorithm procceeds as follows;

1. Initialization - Determine the initial value of parameter $\theta^{(0)}$. Here, ${\bf \theta}^{(0)}=(0.5,0.5)$ this time.

This part gives initial condition of the Markov chain.  Here, $(a^{(0)},b^{(0)})=(0.5,0.5)$ and set the iteration count.

2. For each iteration from here onwards, 
    * **Generate**: Propose a new value for $t^{\text{th}}$ value of $\theta$.
    
    * **Calculate**: Calculate the acceptance probability $\alpha$ of the newly proposed $\theta^{(t)}$.
    
        $$
         \alpha(\theta^{(t-1)},\tilde{\theta}) = \min{\biggl\{1,\frac{q(\tilde{\theta}|\theta^{(t-1)})}{q(\theta^{(t-1)}|\tilde{\theta})}\frac{p(\tilde{\theta})}{p(\theta^{(t-1)})} \biggr\}}
        $$
        
    * **Accept or Reject**: The decicsion whether the new parameter is to be accepted is decided by the following   
    
        $$
        \theta^{(t)}= \begin{cases}
        \tilde{\theta} & (U \leq \alpha(\theta^{(t-1)},\tilde{\theta}) \\
        \theta^{(t-1)} & (U>\alpha(\theta^{(t-1)},\tilde{\theta})).
        \end{cases}
        $$
        where $U$ is a random number drawn from a uniform distribution and $U \in [0,1]$.

The following is done below. This may take a relatively long time depending on your computer.

In the Markov chain sampling, the chain is assumed to converge to an invariant distribution. A distirbution $\bar{p}$ is an invariant distribution of transition distirbution $Q(\theta,\tilde{\theta})$ when

$$
\bar{p}(\tilde{\theta})=\int^{\infty}_{\infty} \bar{p}(\theta)Q(\theta,\tilde{\theta}).
$$

Particulary, the posterior probability $p(\theta|D)$ need to be an invariant distribution for a Markov chain. The invariant distributions is the case when the Markov chain has converged. Therefore, the first several steps are to be throwed out. This fraction of the chain is called **burn-in fraction**. The behavior of the burn-in fraction is dependant on where your initialization points are. 

We can set the number of the steps to be throw out in the box below. A small number may cause to accept posterior distribution that has not converged to an invariant distribution.

Let us plot the posterior values of intercept and slope and see the behavior of the Mrkov chain. We can see the convergence to the mean value (red-dashed line) if MCMC has succeeded.

*(Burn-in region is removed for calculating the mean value of each parameter)*

We know that the posterior distribution is normal. Therefore, we can estimate the mean of the normal by fitting normal distributions to the results from the MCMC. The posterior distribution of the two parameters can be displayed as histograms.

Finally, we can confirm that the obtained parameters using MCMC represent the original distribution of data. We compare our results with th result of linear regression.